In [1]:
import torch
import torchvision
import sys
if '/home/dennis/git_repos/mslp-dataset/benchmark/nn' in sys.path:
    sys.path.remove('/home/dennis/git_repos/mslp-dataset/benchmark/nn')
sys.path.insert(0, '/home/dennis/git_repos/mslp-dataset/benchmark/nn')
from utils.dataset import MARITIMEDETECTION
from models.faster_rcnn import LitFasterRCNN
from utils.transforms import ToTensor
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint
from tqdm import tqdm

### Evaluate Faster RCNN results with Mean Average Precision
* best result with lr: 0.0001

In [2]:
test_dataset = MARITIMEDETECTION("/media/dennis/3B9FC6C559F7A944/converted_object_detection_dataset_imu", "test",
                                    transform=torchvision.transforms.Compose([ 
                                    ToTensor()]))

model = LitFasterRCNN.load_from_checkpoint("/home/dennis/git_repos/mslp-dataset/benchmark/nn/faster_rcnn_log_new/version_5/fasterrcnn-epoch=35-val_loss=0.00.ckpt", map_location="cuda")
model.eval()
metric = MeanAveragePrecision(class_metrics=True)

100%|██████████| 310/310 [00:00<00:00, 407.83it/s]


In [3]:
model = model.cuda()
with torch.no_grad():
    for sample in tqdm(test_dataset):
        img, targets = sample["left_img"], sample["targets"]

        output = model(img.cuda(), None)

        scores = output[0]["scores"].cpu()
        boxes = output[0]["boxes"].cpu()
        labels = output[0]["labels"].cpu()

        score_mask = scores > 0.8
        scores = scores[score_mask]
        boxes = boxes[score_mask]
        labels_int = labels[score_mask]
        labels = [str(l) for l in labels_int.tolist()]

        image = sample["left_img"]
        image = (image*255).to(torch.uint8)

        gt_mask = targets[0].sum(dim=1) != 0
        gt_boxes = targets[0][gt_mask]
        gt_labels_int = targets[1][gt_mask].to(torch.int32).tolist()
        gt_labels = [str(num) for num in gt_labels_int]
        
        preds = [dict(boxes=boxes, scores=scores, labels=labels_int)]
        target = [dict(boxes=gt_boxes, labels=torch.tensor(gt_labels_int))]
        metric.update(preds, target)

pprint(metric.compute())


100%|██████████| 310/310 [00:59<00:00,  5.22it/s]


{'classes': tensor([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32),
 'map': tensor(0.2825),
 'map_50': tensor(0.5706),
 'map_75': tensor(0.2440),
 'map_large': tensor(0.3003),
 'map_medium': tensor(0.1951),
 'map_per_class': tensor([ 0.2630,  0.4367,  0.3048,  0.0069,  0.3456,  0.1561, -1.0000,  0.4291,
         0.3178]),
 'map_small': tensor(0.),
 'mar_1': tensor(0.3336),
 'mar_10': tensor(0.3585),
 'mar_100': tensor(0.3585),
 'mar_100_per_class': tensor([ 0.3642,  0.5552,  0.4552,  0.0067,  0.3561,  0.1610, -1.0000,  0.5862,
         0.3837]),
 'mar_large': tensor(0.3491),
 'mar_medium': tensor(0.2727),
 'mar_small': tensor(0.)}
